# Battle of the Neighborhoods ( Toronto )

## Business problem statement:
Since Canada has become on the spotlight of young professionals during the last years because of the benefits offered to skilled professionals, my wife and I decided to explore the idea of moving to Toronto looking for quitter life and eventually establish a family, so far we are in the process of paperwork and being optimistic we should be travelling in a couple of months.

However the consequences of a big decision like this involves many aspects to cover, so, with the help of this course, I am going to tackle a problem that come to my mind recently:

**How do we choose the best neighborhood to initially move?**

Given that we do not have previous knowledge of the City, by “initially” I mean that our plan is to rent a place for a couple of months, then eventually move to an area near to our jobs and study centers.

In order to narrow the scope, the neighborhood should meet the following requirements:

* Easy access to public transportation
* Lot of restaurants, since we are not keen on cooking
* Low Crime rates 
* Affordable Rent Prices 
* Things to do in the cornerstones
* Some night life should be welcome



## Required data

1.	Neighborhoods in Toronto - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
2.	Foursquare data
3.	Neighborhoods Crime data in Toronto - http://data.torontopolice.on.ca/datasets/af500b5abb7240399853b35a2362d0c0_0?geometry=-79.812%2C43.632%2C-78.944%2C43.805&selectedAttribute=Total_Rate 
4.	AirBnb Prices prices - http://insideairbnb.com/get-the-data.html 


## Machine Learning approach

Overall, I will use the data available in the sources to perform a clustering based analysis to find the group that better meet the initial requirements.

It is expected to find one or more clusters with neighborhoods with crime rates under the average, rent prices around the average, with lots of restaurants and with bus/train stations.

Finally, once plotted in a map, it should be easier to decide which cluster is near to the university we are going to study.


## Results

## Summary and Conclusions

In [4]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 7.1MB/s ta 0:00:01
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

In [5]:
import geocoder # import geocoder

In [ ]:
# initialize your variable to None
lat_lng_coords = None
postal_code = 'M5P'

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [47]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [51]:
#Use geopy library to get the latitude and longitude values of New York City.
address = 'Forest Hill South,Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Forest Hill South,Toronto, ON are 43.6935586, -79.4139023.


In [53]:
location

Location(Forest Hill, Old Toronto, Toronto, Ontario, M5P2W8, Canada, (43.6935586, -79.4139023, 0.0))

In [88]:
def getLocationByAddress(address):
    address = address + ', Toronto, Ontario'
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    return(location)

def getLatituteByLocation(location):
    if location is None:
        return None
    return(location.latitude)

def getLongitudeByLocation(location):
    if location is None:
        return None
    return(location.longitude)

In [89]:
#location_result = getLocationByAddress('Forest Hill South,Toronto, ON')
location_result = getLocationByAddress('Forest Hill South')

In [90]:
print(location_result.latitude,location_result.longitude)

43.6935586 -79.4139023


In [91]:
print( getLatituteByLocation(location_result) , getLongitudeByLocation(location_result) )

43.6935586 -79.4139023


In [92]:
# The code was removed by Watson Studio for sharing.

,Neighbourhood_Formatted,Total_Rate
0,Mount Pleasant East,226.845357
1,Markland Wood,337.818494
2,Yonge-Eglinton,412.358054
3,Pleasant View,435.878584
4,Caledonia-Fairbank,438.229039


In [93]:
df_CrimeRate['location'] = df_CrimeRate['Neighbourhood_Formatted'].apply(getLocationByAddress)

# Save the result in a new column
#df['salary'] = df['SAL-RATE'].apply(money_to_float)

In [94]:
df_CrimeRate['Latitute'] = df_CrimeRate['location'].apply(getLatituteByLocation)
df_CrimeRate['Longitude'] = df_CrimeRate['location'].apply(getLongitudeByLocation)
df_CrimeRate

,Neighbourhood_Formatted,Total_Rate,location,Latitute,Longitude
0,Mount Pleasant East,226.845357,"(Mount Pleasant, 1, Eglinton Avenue East, Davi...",43.708417,-79.390135
1,Markland Wood,337.818494,"(Markland Wood Golf Club, 245, Markland Drive,...",43.631238,-79.585437
2,Yonge-Eglinton,412.358054,"(Yonge & Eglinton, Old Toronto, Toronto, Ontar...",43.706748,-79.398327
3,Pleasant View,435.878584,"(Pleasant View, North York, Toronto, Ontario, ...",43.787048,-79.333714
4,Caledonia-Fairbank,438.229039,"(Caledonia Road, Fairbank, York, Toronto, Onta...",43.698951,-79.463611
5,Tam O'Shanter-Sullivan,450.852500,None,NaN,NaN
6,Banbury-Don Mills,454.500786,"(Banbury Road, The Bridle Path, North York, To...",43.734804,-79.357243
7,High Park-Swansea,467.289720,"(high park, 1, Colborne Lodge Drive, High Park...",43.645593,-79.465320
8,Leaside-Bennington,477.644278,None,NaN,NaN
9,Pelmo Park-Humberlea,522.445043,"(Pelmo Park Public School, 180, Gary Drive, Hu...",43.712195,-79.517511


In [95]:
#df_CrimeRate.to_csv('df_CrimeRate.csv', sep='\t', encoding='utf-8')

In [96]:
df_CrimeRate[df_CrimeRate['location'].isnull()]

,Neighbourhood_Formatted,Total_Rate,location,Latitute,Longitude
5,Tam O'Shanter-Sullivan,450.852500,None,NaN,NaN
8,Leaside-Bennington,477.644278,None,NaN,NaN
19,Westminster-Branson,596.650133,None,NaN,NaN
26,Humewood-Cedarvale,665.228335,None,NaN,NaN
33,Thistletown-Beaumond Heights,748.176320,None,NaN,NaN
34,Rockcliffe-Smythe,754.408289,None,NaN,NaN
37,Willowridge-Martingrove-Richview,793.621901,None,NaN,NaN
38,Weston-Pellam Park,795.939555,None,NaN,NaN
39,Bridle Path-Sunnybrook-York Mills,825.351539,None,NaN,NaN
48,Brookhaven-Amesbury,871.598639,None,NaN,NaN


In [99]:
address = "Tam O'Shanter"
location_result = getLocationByAddress(address)
address

"Tam O'Shanter"

In [103]:
address = "Leaside-Bennington, Toronto, Ontario"
geolocator = Nominatim()
location = geolocator.geocode(address)
location